In [2]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from vit_pytorch.efficient import ViT

from vit_pytorch import vit_b_16 as vit

import scipy.misc

import torchvision.models as models
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,accuracy_score
import requests
from torch.autograd import Function
from tqdm.notebook import tqdm

In [3]:
import torch
import os
import numpy as np
import random
seed = 66 #66 
torch.manual_seed(seed)  # 为CPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
if torch.cuda.is_available():
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子

In [4]:
print(f"Torch: {torch.__version__}")

Torch: 1.12.0+cu113


In [5]:
# Training settings
batch_size = 32
epochs = 30
lr = 1e-5
gamma = 0.7

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Data

In [8]:
os.makedirs('data', exist_ok=True)

In [9]:
train_dir_feature = './PSD feature/train'
test_dir_feature = './PSD feature/test'

In [11]:
train_list_feature = glob.glob(os.path.join(train_dir_feature,'*.npy'))
test_list_feature = glob.glob(os.path.join(test_dir_feature, '*.npy'))

In [13]:
print(f"Train_feature Data: {len(train_list_feature)}")
print(f"Test_feature Data: {len(test_list_feature)}")

Train_feature Data: 152730
Test_feature Data: 0


In [15]:
labels_feature = [path.split('/')[-1].split('.')[0] for path in train_list_feature]

## Split

In [17]:
train_list_feature, valid_list_feature = train_test_split(train_list_feature, 
                                          test_size=0.4,
                                          stratify=labels_feature,
                                          random_state=seed)

In [21]:
train_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


## Load Datasets

In [22]:
class Dataset(Dataset):
    def __init__(self, file_list_f, transform=None):
        self.file_list_f = file_list_f

        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list_f)
        return self.filelength

    def __getitem__(self, idx):
        img_path_f = self.file_list_f[idx]

        img_f = np.load(img_path_f)

        img_transformed_f = self.transform(img_f)


        label_f = img_path_f.split("/")[-1].split(".")[0]
        
        label_f = int(label_f)

        return img_transformed_f, label_f


In [23]:
train_data = Dataset(train_list_feature,train_list_temporal, transform=train_transforms)
valid_data = Dataset(valid_list_feature,valid_list_temporal, transform=test_transforms)

In [24]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)

In [25]:
print(len(train_data), len(train_loader))

91638 2864


In [26]:
print(len(valid_data), len(valid_loader))

61092 1910


In [27]:
model_feature  = vit().to(device)

In [28]:
conv1 = nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3, bias=False) 
model_ResNet = models.resnet18(pretrained=False)
model_ResNet=nn.Sequential(conv1,*list(model_ResNet.children())[1:-1])
model_ResNet.conv1 = conv1

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [29]:
class model_fussion(nn.Module):
    def __init__(
        self,
        model_feature,
        model_ResNet
        
        

    ):
        super().__init__()
        self.model_feature = model_feature #ViT
        self.model_temporal = model_ResNet #CNN

        features_CNN = list(model_ResNet.children())
        self.enc_CNN_1 = nn.Sequential(*features_CNN[:5])
        self.enc_CNN_2 = nn.Sequential(*features_CNN[5:6])
        self.enc_CNN_3 = nn.Sequential(*features_CNN[6:7])
        self.enc_CNN_4 = nn.Sequential(*features_CNN[7:])
        
        self.dense = nn.Linear(768+512,3)
        self.dense_cnntovit_1 = nn.Linear(64,768)#cnn to vit 1
        self.dense_cnntovit_2 = nn.Linear(128,768)#cnn to vit 2
        self.dense_cnntovit_3 = nn.Linear(256,768)#cnn to vit 3
        
        self.cnn_vittocnn_1 = nn.Conv2d(64+768,64,1)#vit to cnn 1
        self.cnn_vittocnn_2 = nn.Conv2d(128+768,128,1)#vit to cnn 2
        self.cnn_vittocnn_3 = nn.Conv2d(256+768,256,1)#vit to cnn 3

        
    def forward(self,x1,x2):
        
        #Stage1
        x_stage1 = self.model_feature(x1,stage=1)
        x_cnn_1 = self.enc_CNN_1(x2)
        
        x_cnn_1_temp = torch.mean(x_cnn_1,dim=-1).mean(dim=-1)
        x_cnn_1_temp = self.dense_cnntovit_1(x_cnn_1_temp)
        x_cnn_1_temp = x_cnn_1_temp.expand(x_cnn_1_temp.shape[0],-1,768)
        x_stage1_fuss = torch.concat([x_stage1,x_cnn_1_temp],dim=1)
        
        x_stage1_temp = x_stage1[:,0,:].squeeze()
        x_stage1_temp = x_stage1_temp.unsqueeze(-1).unsqueeze(-1).expand(x_stage1_temp.shape[0],768,8,8)
        x_cnn_1_fuss = torch.concat([x_cnn_1,x_stage1_temp],dim=1)
        x_cnn_1_fuss = self.cnn_vittocnn_1(x_cnn_1_fuss)
        
        #Stage2
        x_stage2 = self.model_feature(x_stage1_fuss,stage=2)
        x_cnn_2 = self.enc_CNN_2(x_cnn_1_fuss)
        
        x_cnn_2_temp = torch.mean(x_cnn_2,dim=-1).mean(dim=-1)
        x_cnn_2_temp = self.dense_cnntovit_2(x_cnn_2_temp)
        x_cnn_2_temp = x_cnn_2_temp.expand(x_cnn_2_temp.shape[0],-1,768)
        x_stage2_fuss = torch.concat([x_stage2,x_cnn_2_temp],dim=1)
        
        x_stage2_temp = x_stage2[:,0,:].squeeze()
        x_stage2_temp = x_stage2_temp.unsqueeze(-1).unsqueeze(-1).expand(x_stage2_temp.shape[0],768,4,4)
        x_cnn_2_fuss = torch.concat([x_cnn_2,x_stage2_temp],dim=1)
        x_cnn_2_fuss = self.cnn_vittocnn_2(x_cnn_2_fuss)
        
        #Stage3
        x_stage3 = self.model_feature(x_stage2_fuss,stage=3)
        x_cnn_3 = self.enc_CNN_3(x_cnn_2_fuss)
        
        x_cnn_3_temp = torch.mean(x_cnn_3,dim=-1).mean(dim=-1)
        x_cnn_3_temp = self.dense_cnntovit_3(x_cnn_3_temp)
        x_cnn_3_temp = x_cnn_3_temp.expand(x_cnn_3_temp.shape[0],-1,768)
        x_stage3_fuss = torch.concat([x_stage3,x_cnn_3_temp],dim=1)
        
        x_stage3_temp = x_stage3[:,0,:].squeeze()
        x_stage3_temp = x_stage3_temp.unsqueeze(-1).unsqueeze(-1).expand(x_stage3_temp.shape[0],768,2,2)
        x_cnn_3_fuss = torch.concat([x_cnn_3,x_stage3_temp],dim=1)
        x_cnn_3_fuss = self.cnn_vittocnn_3(x_cnn_3_fuss)
        
        #Stage4
        x_stage4 = self.model_feature(x_stage3_fuss,stage=4)
        x_cnn_4 = self.enc_CNN_4(x_cnn_3_fuss)

        x2 = x_cnn_4.squeeze(-1).squeeze(-1)
        out = self.dense(torch.concat([x_stage4,x2],dim=-1))
        return out

In [30]:
model = model_fussion(model_feature,model_ResNet).to(device)

### Training

In [31]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

best_acc = 0 

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data_f,label in tqdm(train_loader):
        
        data_f = data_f.to(device)
        
        label = label.to(device)
        
        data = data_f

        output = model(data,data)
        
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        pred = []
        labellist = []
        for data_f, label in valid_loader:
            data_f = data_f.to(device)
            
            label = label.to(device)
            
            data = data_f

            val_output = model(data,data)
            val_loss = criterion(val_output, label)
            
            pred_batch = list(val_output.argmax(dim=1).squeeze(-1).detach().cpu().numpy())
            pred.extend(pred_batch)
            
            label_batch = list(label.detach().cpu().numpy())
            labellist.extend(label_batch)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    log = open("/hy-tmp/CIT.txt", mode = "a", encoding = "utf-8")
    
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    ,file = log)
    print(len(pred),len(labellist),file = log)
    print(confusion_matrix(labellist, pred, labels=[0, 1, 2]),file = log)
    print(lr,file = log)
    
    log.close()

  0%|          | 0/2864 [00:00<?, ?it/s]

save


  0%|          | 0/2864 [00:00<?, ?it/s]

save


  0%|          | 0/2864 [00:00<?, ?it/s]